# UDA by Backpropogation

## Adds a Gradient Reversal Layer and a Discriminator (Classification Head) on top of YOLOv3

### Part 1: Gradient Reversal Layer

Initialize Gradient Reversal class

In [1]:
import torch
from torch import nn
from pytorch_metric_learning.utils import common_functions as pml_cf

/home/eranario/miniconda3/envs/yolo-uda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GradientReversal(torch.nn.Module):
    """
    Implementation of the gradient reversal layer described in
    [Domain-Adversarial Training of Neural Networks](https://arxiv.org/abs/1505.07818),
    which 'leaves the input unchanged during forward propagation
    and reverses the gradient by multiplying it
    by a negative scalar during backpropagation.'
    """

    def __init__(self, alpha: float = 1.0):
        """
        Arguments:
            weight: The gradients  will be multiplied by ```-alpha```
                during the backward pass.
        """
        super().__init__()
        self.register_buffer("alpha", torch.tensor([alpha]))
        pml_cf.add_to_recordable_attributes(self, "alpha")

    def update_weight(self, new_alpha):
        self.weight[0] = new_alpha

    def forward(self, x):
        """"""
        return _GradientReversal.apply(x, pml_cf.to_device(self.alpha, x))

    def extra_repr(self, delimiter="\n"):
        """"""
        return delimiter.join([f"{a}=str{getattr(self, a)}" for a in ["alpha"]])
        # return c_f.extra_repr(self, ["weight"])


class _GradientReversal(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, alpha):
        ctx.alpha = alpha
        return input

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.alpha * grad_output, None

Test it

In [3]:
import copy

def test_grad_reverese(alpha):
    criterion = torch.nn.MSELoss()
    input, output = torch.randn(8, 5), torch.randn(8, 1)

    network = nn.Sequential(nn.Linear(5, 1), torch.nn.Linear(1, 1))
    revnetwork = nn.Sequential(
        copy.deepcopy(network), 
        GradientReversal(alpha),
        )

    criterion(network(input), output).backward()
    criterion(revnetwork(input), output).backward()

    for p1, p2 in zip(network.parameters(), revnetwork.parameters()):
      print("-"*100)
      print(f"original: {p1.grad}")
      print(f"reversed: {p2.grad/alpha}")
      assert torch.isclose(p1.grad, ((-p2.grad)/alpha)).all()
    print("-"*100)

test_grad_reverese(1)

----------------------------------------------------------------------------------------------------
original: tensor([[ 0.0636, -0.0822,  0.0423, -0.0366,  0.2136]])
reversed: tensor([[-0.0636,  0.0822, -0.0423,  0.0366, -0.2136]])
----------------------------------------------------------------------------------------------------
original: tensor([-0.1749])
reversed: tensor([0.1749])
----------------------------------------------------------------------------------------------------
original: tensor([[-0.4986]])
reversed: tensor([[0.4986]])
----------------------------------------------------------------------------------------------------
original: tensor([-1.7136])
reversed: tensor([1.7136])
----------------------------------------------------------------------------------------------------


### Part 2: Initialize YOLOv3 baseline architecture

In [4]:
from __future__ import division

import os
from itertools import chain
from typing import List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from pytorchyolo.utils.parse_config import parse_model_config
from pytorchyolo.utils.utils import weights_init_normal

In [5]:
def create_modules(module_defs: List[dict]) -> Tuple[dict, nn.ModuleList]:
    """
    Constructs module list of layer blocks from module configuration in module_defs

    :param module_defs: List of dictionaries with module definitions
    :return: Hyperparameters and pytorch module list
    """
    hyperparams = module_defs.pop(0)
    hyperparams.update({
        'batch': int(hyperparams['batch']),
        'subdivisions': int(hyperparams['subdivisions']),
        'width': int(hyperparams['width']),
        'height': int(hyperparams['height']),
        'channels': int(hyperparams['channels']),
        'optimizer': hyperparams.get('optimizer'),
        'momentum': float(hyperparams['momentum']),
        'decay': float(hyperparams['decay']),
        'learning_rate': float(hyperparams['learning_rate']),
        'burn_in': int(hyperparams['burn_in']),
        'max_batches': int(hyperparams['max_batches']),
        'policy': hyperparams['policy'],
        'lr_steps': list(zip(map(int,   hyperparams["steps"].split(",")),
                             map(float, hyperparams["scales"].split(","))))
    })
    assert hyperparams["height"] == hyperparams["width"], \
        "Height and width should be equal! Non square images are padded with zeros."
    output_filters = [hyperparams["channels"]]
    module_list = nn.ModuleList()
    for module_i, module_def in enumerate(module_defs):
        modules = nn.Sequential()

        if module_def["type"] == "convolutional":
            bn = int(module_def["batch_normalize"])
            filters = int(module_def["filters"])
            kernel_size = int(module_def["size"])
            pad = (kernel_size - 1) // 2
            modules.add_module(
                f"conv_{module_i}",
                nn.Conv2d(
                    in_channels=output_filters[-1],
                    out_channels=filters,
                    kernel_size=kernel_size,
                    stride=int(module_def["stride"]),
                    padding=pad,
                    bias=not bn,
                ),
            )
            if bn:
                modules.add_module(f"batch_norm_{module_i}",
                                   nn.BatchNorm2d(filters, momentum=0.1, eps=1e-5))
            if module_def["activation"] == "leaky":
                modules.add_module(f"leaky_{module_i}", nn.LeakyReLU(0.1))
            elif module_def["activation"] == "mish":
                modules.add_module(f"mish_{module_i}", nn.Mish())
            elif module_def["activation"] == "logistic":
                modules.add_module(f"sigmoid_{module_i}", nn.Sigmoid())
            elif module_def["activation"] == "swish":
                modules.add_module(f"swish_{module_i}", nn.SiLU())

        elif module_def["type"] == "maxpool":
            kernel_size = int(module_def["size"])
            stride = int(module_def["stride"])
            if kernel_size == 2 and stride == 1:
                modules.add_module(f"_debug_padding_{module_i}", nn.ZeroPad2d((0, 1, 0, 1)))
            maxpool = nn.MaxPool2d(kernel_size=kernel_size, stride=stride,
                                   padding=int((kernel_size - 1) // 2))
            modules.add_module(f"maxpool_{module_i}", maxpool)

        elif module_def["type"] == "upsample":
            upsample = Upsample(scale_factor=int(module_def["stride"]), mode="nearest")
            modules.add_module(f"upsample_{module_i}", upsample)

        elif module_def["type"] == "route":
            layers = [int(x) for x in module_def["layers"].split(",")]
            filters = sum([output_filters[1:][i] for i in layers]) // int(module_def.get("groups", 1))
            modules.add_module(f"route_{module_i}", nn.Sequential())

        elif module_def["type"] == "shortcut":
            filters = output_filters[1:][int(module_def["from"])]
            modules.add_module(f"shortcut_{module_i}", nn.Sequential())

        elif module_def["type"] == "yolo":
            anchor_idxs = [int(x) for x in module_def["mask"].split(",")]
            # Extract anchors
            anchors = [int(x) for x in module_def["anchors"].split(",")]
            anchors = [(anchors[i], anchors[i + 1]) for i in range(0, len(anchors), 2)]
            anchors = [anchors[i] for i in anchor_idxs]
            num_classes = int(module_def["classes"])
            new_coords = bool(module_def.get("new_coords", False))
            # Define detection layer
            yolo_layer = YOLOLayer(anchors, num_classes, new_coords)
            modules.add_module(f"yolo_{module_i}", yolo_layer)
        # Register module list and number of output filters
        module_list.append(modules)
        output_filters.append(filters)

    return hyperparams, module_list

In [6]:
class Upsample(nn.Module):
    """ nn.Upsample is deprecated """

    def __init__(self, scale_factor, mode: str = "nearest"):
        super(Upsample, self).__init__()
        self.scale_factor = scale_factor
        self.mode = mode

    def forward(self, x):
        x = F.interpolate(x, scale_factor=self.scale_factor, mode=self.mode)
        return x

In [7]:
class YOLOLayer(nn.Module):
    """Detection layer"""

    def __init__(self, anchors: List[Tuple[int, int]], num_classes: int, new_coords: bool):
        """
        Create a YOLO layer

        :param anchors: List of anchors
        :param num_classes: Number of classes
        :param new_coords: Whether to use the new coordinate format from YOLO V7
        """
        super(YOLOLayer, self).__init__()
        self.num_anchors = len(anchors)
        self.num_classes = num_classes
        self.new_coords = new_coords
        self.mse_loss = nn.MSELoss()
        self.bce_loss = nn.BCELoss()
        self.no = num_classes + 5  # number of outputs per anchor
        self.grid = torch.zeros(1)  # TODO

        anchors = torch.tensor(list(chain(*anchors))).float().view(-1, 2)
        self.register_buffer('anchors', anchors)
        self.register_buffer(
            'anchor_grid', anchors.clone().view(1, -1, 1, 1, 2))
        self.stride = None

    def forward(self, x: torch.Tensor, img_size: int) -> torch.Tensor:
        """
        Forward pass of the YOLO layer

        :param x: Input tensor
        :param img_size: Size of the input image
        """
        stride = img_size // x.size(2)
        self.stride = stride
        bs, _, ny, nx = x.shape  # x(bs,255,20,20) to x(bs,3,20,20,85)
        x = x.view(bs, self.num_anchors, self.no, ny, nx).permute(0, 1, 3, 4, 2).contiguous()

        if not self.training:  # inference
            if self.grid.shape[2:4] != x.shape[2:4]:
                self.grid = self._make_grid(nx, ny).to(x.device)

            if self.new_coords:
                x[..., 0:2] = (x[..., 0:2] + self.grid) * stride  # xy
                x[..., 2:4] = x[..., 2:4] ** 2 * (4 * self.anchor_grid) # wh
            else:
                x[..., 0:2] = (x[..., 0:2].sigmoid() + self.grid) * stride  # xy
                x[..., 2:4] = torch.exp(x[..., 2:4]) * self.anchor_grid # wh
                x[..., 4:] = x[..., 4:].sigmoid() # conf, cls
            x = x.view(bs, -1, self.no)

        return x

    @staticmethod
    def _make_grid(nx: int = 20, ny: int = 20) -> torch.Tensor:
        """
        Create a grid of (x, y) coordinates

        :param nx: Number of x coordinates
        :param ny: Number of y coordinates
        """
        yv, xv = torch.meshgrid([torch.arange(ny), torch.arange(nx)], indexing='ij')
        return torch.stack((xv, yv), 2).view((1, 1, ny, nx, 2)).float()

In [8]:
class Darknet(nn.Module):
    """YOLOv3 object detection model"""

    def __init__(self, config_path):
        super(Darknet, self).__init__()
        self.module_defs = parse_model_config(config_path)
        self.hyperparams, self.module_list = create_modules(self.module_defs)
        self.yolo_layers = [layer[0]
                            for layer in self.module_list if isinstance(layer[0], YOLOLayer)]
        self.seen = 0
        self.header_info = np.array([0, 0, 0, self.seen, 0], dtype=np.int32)

    def forward(self, x):
        feature_maps = [] # save feature maps for discriminator
        img_size = x.size(2)
        layer_outputs, yolo_outputs = [], []
        for i, (module_def, module) in enumerate(zip(self.module_defs, self.module_list)):
            if module_def["type"] in ["convolutional", "upsample", "maxpool"]:
                x = module(x)
            elif module_def["type"] == "route":
                combined_outputs = torch.cat([layer_outputs[int(layer_i)] for layer_i in module_def["layers"].split(",")], 1)
                group_size = combined_outputs.shape[1] // int(module_def.get("groups", 1))
                group_id = int(module_def.get("group_id", 0))
                x = combined_outputs[:, group_size * group_id : group_size * (group_id + 1)] # Slice groupings used by yolo v4
            elif module_def["type"] == "shortcut":
                layer_i = int(module_def["from"])
                x = layer_outputs[-1] + layer_outputs[layer_i]
            elif module_def["type"] == "yolo":
                x = module[0](x, img_size)
                yolo_outputs.append(x)
            layer_outputs.append(x)
            
            if i in [81, 93, 105]: # i starts at 0
                feature_maps.append(x)
        return [yolo_outputs, feature_maps] if self.training else torch.cat(yolo_outputs, 1)

    def load_darknet_weights(self, weights_path):
        """Parses and loads the weights stored in 'weights_path'"""

        # Open the weights file
        with open(weights_path, "rb") as f:
            # First five are header values
            header = np.fromfile(f, dtype=np.int32, count=5)
            self.header_info = header  # Needed to write header when saving weights
            self.seen = header[3]  # number of images seen during training
            weights = np.fromfile(f, dtype=np.float32)  # The rest are weights

        # Establish cutoff for loading backbone weights
        cutoff = None
        # If the weights file has a cutoff, we can find out about it by looking at the filename
        # examples: darknet53.conv.74 -> cutoff is 74
        filename = os.path.basename(weights_path)
        if ".conv." in filename:
            try:
                cutoff = int(filename.split(".")[-1])  # use last part of filename
            except ValueError:
                pass

        ptr = 0
        for i, (module_def, module) in enumerate(zip(self.module_defs, self.module_list)):
            if i == cutoff:
                break
            if module_def["type"] == "convolutional":
                conv_layer = module[0]
                if module_def["batch_normalize"]:
                    # Load BN bias, weights, running mean and running variance
                    bn_layer = module[1]
                    num_b = bn_layer.bias.numel()  # Number of biases
                    # Bias
                    bn_b = torch.from_numpy(
                        weights[ptr: ptr + num_b]).view_as(bn_layer.bias)
                    bn_layer.bias.data.copy_(bn_b)
                    ptr += num_b
                    # Weight
                    bn_w = torch.from_numpy(
                        weights[ptr: ptr + num_b]).view_as(bn_layer.weight)
                    bn_layer.weight.data.copy_(bn_w)
                    ptr += num_b
                    # Running Mean
                    bn_rm = torch.from_numpy(
                        weights[ptr: ptr + num_b]).view_as(bn_layer.running_mean)
                    bn_layer.running_mean.data.copy_(bn_rm)
                    ptr += num_b
                    # Running Var
                    bn_rv = torch.from_numpy(
                        weights[ptr: ptr + num_b]).view_as(bn_layer.running_var)
                    bn_layer.running_var.data.copy_(bn_rv)
                    ptr += num_b
                else:
                    # Load conv. bias
                    num_b = conv_layer.bias.numel()
                    conv_b = torch.from_numpy(
                        weights[ptr: ptr + num_b]).view_as(conv_layer.bias)
                    conv_layer.bias.data.copy_(conv_b)
                    ptr += num_b
                # Load conv. weights
                num_w = conv_layer.weight.numel()
                conv_w = torch.from_numpy(
                    weights[ptr: ptr + num_w]).view_as(conv_layer.weight)
                conv_layer.weight.data.copy_(conv_w)
                ptr += num_w

    def save_darknet_weights(self, path, cutoff=-1):
        """
            @:param path    - path of the new weights file
            @:param cutoff  - save layers between 0 and cutoff (cutoff = -1 -> all are saved)
        """
        fp = open(path, "wb")
        self.header_info[3] = self.seen
        self.header_info.tofile(fp)

        # Iterate through layers
        for i, (module_def, module) in enumerate(zip(self.module_defs[:cutoff], self.module_list[:cutoff])):
            if module_def["type"] == "convolutional":
                conv_layer = module[0]
                # If batch norm, load bn first
                if module_def["batch_normalize"]:
                    bn_layer = module[1]
                    bn_layer.bias.data.cpu().numpy().tofile(fp)
                    bn_layer.weight.data.cpu().numpy().tofile(fp)
                    bn_layer.running_mean.data.cpu().numpy().tofile(fp)
                    bn_layer.running_var.data.cpu().numpy().tofile(fp)
                # Load conv bias
                else:
                    conv_layer.bias.data.cpu().numpy().tofile(fp)
                # Load conv weights
                conv_layer.weight.data.cpu().numpy().tofile(fp)

        fp.close()

### Part 3: Initialize Driscriminator

In [9]:
class Discriminator(nn.Module):
    """
    A 3-layer MLP + Greadient Reversal Layer for domain classification.
    """

    def __init__(self, in_size=52, h=2048, out_size=1, alpha=1.0):
        """
        Arguments:
            in_size: size of the input
            h: hidden layer size
            out_size: size of the output
            alpha: grl constant
        """

        super().__init__()
        self.h = h
        self.net = nn.Sequential(
            GradientReversal(alpha=alpha),
            nn.Linear(in_size, h),
            nn.ReLU(),
            nn.Linear(h, h),
            nn.ReLU(),
            nn.Linear(h, out_size),
        )
        self.out_size = out_size

    def forward(self, x):
        """"""
        return self.net(x).squeeze(1)

### Part 4: Load model, create dataloader and optimizer

In [10]:
import tqdm
import torch.optim as optim

from torch.utils.data import Dataset
from pytorchyolo.utils.datasets import ListDataset
from pytorchyolo.utils.augmentations import AUGMENTATION_TRANSFORMS
from torch.utils.data import DataLoader
from pytorchyolo.test import _create_validation_data_loader
from pytorchyolo.utils.loss import compute_loss
from pytorchyolo.utils.utils import to_cpu, worker_seed_set, ap_per_class, get_batch_statistics, non_max_suppression, xywh2xyxy

In [11]:
PATH_TO_CFG = "/group/jmearlesgrp/scratch/eranario/CropGAN/yolo_uda/configs/yolov3.cfg"
PATH_TO_PRETRAINED = "/group/jmearlesgrp/data/yolo_grl_data/weights/yolov3.weights"
PATH_TO_TRAIN = "/group/jmearlesgrp/data/yolo_grl_data/BordenNight/source/train/train.txt"
# PATH_TO_VAL = "/group/jmearlesgrp/data/yolo_grl_data/BordenNight/source/val/val.txt"
PATH_TO_VAL = '/group/jmearlesgrp/data/yolo_grl_data/BordenNight/target/target.txt'
PATH_TO_TARGET = "/group/jmearlesgrp/data/yolo_grl_data/BordenNight/target/images"
EPOCHS = 300
N_CPU = 6
CHECKPOINT_INTERVAL = 10
EVALUATE_INTERVAL = 1
IOU_THRESH = 0.5
CONF_THRESH = 0.1
NMS_THRESH = 0.5
alpha = 1.0
VERBOSE = False
CLASS_NAMES = ["0"]

In [12]:
def load_model(model_path, weights_path=None):
    """Loads the yolo model from file.

    :param model_path: Path to model definition file (.cfg)
    :type model_path: str
    :param weights_path: Path to weights or checkpoint file (.weights or .pth)
    :type weights_path: str
    :return: Returns model
    :rtype: Darknet
    """
    device = torch.device("cuda" if torch.cuda.is_available()
                          else "cpu")  # Select device for inference
    model = Darknet(model_path).to(device)

    model.apply(weights_init_normal)

    # If pretrained weights are specified, start from checkpoint or weight file
    if weights_path:
        if weights_path.endswith(".pth"):
            # Load checkpoint weights
            model.load_state_dict(torch.load(weights_path, map_location=device))
        else:
            # Load darknet weights
            model.load_darknet_weights(weights_path)
    return model


In [13]:
def _create_data_loader(img_path, batch_size, img_size, n_cpu, multiscale_training=False):
    """Creates a DataLoader for training.

    :param img_path: Path to file containing all paths to training images.
    :type img_path: str
    :param batch_size: Size of each image batch
    :type batch_size: int
    :param img_size: Size of each image dimension for yolo
    :type img_size: int
    :param n_cpu: Number of cpu threads to use during batch generation
    :type n_cpu: int
    :param multiscale_training: Scale images to different sizes randomly
    :type multiscale_training: bool
    :return: Returns DataLoader
    :rtype: DataLoader
    """
    dataset = ListDataset(
        img_path,
        img_size=img_size,
        multiscale=multiscale_training,
        transform=AUGMENTATION_TRANSFORMS)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=n_cpu,
        # num_workers=0,
        pin_memory=True,
        collate_fn=dataset.collate_fn,
        worker_init_fn=worker_seed_set)
    return dataloader

In [14]:
# create model
model = load_model(PATH_TO_CFG, PATH_TO_PRETRAINED)
discriminator = Discriminator(alpha=alpha)
mini_batch_size = model.hyperparams['batch'] // model.hyperparams['subdivisions']

In [15]:
# create dataloader
dataloader = _create_data_loader(
    PATH_TO_TRAIN,
    mini_batch_size,
    model.hyperparams['height'],
    N_CPU,
    multiscale_training=False)

# load validation dataloader
validation_dataloader = _create_validation_data_loader(
    PATH_TO_VAL,
    mini_batch_size,
    model.hyperparams['height'],
    N_CPU)

In [16]:
# create optimizer
params = [p for p in model.parameters() if p.requires_grad]

if (model.hyperparams['optimizer'] in [None, "adam"]):
    optimizer = optim.Adam(
        params,
        lr=model.hyperparams['learning_rate'],
        weight_decay=model.hyperparams['decay'],
    )
elif (model.hyperparams['optimizer'] == "sgd"):
    optimizer = optim.SGD(
        params,
        lr=model.hyperparams['learning_rate'],
        weight_decay=model.hyperparams['decay'],
        momentum=model.hyperparams['momentum'])
else:
    print("Unknown optimizer. Please choose between (adam, sgd).")

### Part 5: Training Loop

In [17]:
from torchvision import transforms
from PIL import Image
import random
import wandb
from terminaltables import AsciiTable
from torch.autograd import Variable

In [18]:
cross_entropy = nn.CrossEntropyLoss()

In [19]:
def print_eval_stats(metrics_output, class_names, verbose):
    if metrics_output is not None:
        precision, recall, AP, f1, ap_class = metrics_output
        if verbose:
            # Prints class AP and mean AP
            ap_table = [["Index", "Class", "AP"]]
            for i, c in enumerate(ap_class):
                ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
            print(AsciiTable(ap_table).table)
        print(f"---- mAP {AP.mean():.5f} ----")
    else:
        print("---- mAP not measured (no detections found by model) ----")

In [20]:
def _evaluate(model, dataloader, class_names, img_size, iou_thres, conf_thres, nms_thres, verbose):
    """Evaluate model on validation dataset.

    :param model: Model to evaluate
    :type model: models.Darknet
    :param dataloader: Dataloader provides the batches of images with targets
    :type dataloader: DataLoader
    :param class_names: List of class names
    :type class_names: [str]
    :param img_size: Size of each image dimension for yolo
    :type img_size: int
    :param iou_thres: IOU threshold required to qualify as detected
    :type iou_thres: float
    :param conf_thres: Object confidence threshold
    :type conf_thres: float
    :param nms_thres: IOU threshold for non-maximum suppression
    :type nms_thres: float
    :param verbose: If True, prints stats of model
    :type verbose: bool
    :return: Returns precision, recall, AP, f1, ap_class
    """
    model.eval()  # Set model to evaluation mode
    print(f"model training: {model.training}")

    Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    labels = []
    sample_metrics = []  # List of tuples (TP, confs, pred)
    for _, imgs, targets in tqdm.tqdm(dataloader, desc="Validating"):
        # Extract labels
        labels += targets[:, 1].tolist()
        # Rescale target
        targets[:, 2:] = xywh2xyxy(targets[:, 2:])
        targets[:, 2:] *= img_size

        imgs = Variable(imgs.type(Tensor), requires_grad=False)

        with torch.no_grad():
            outputs = model(imgs)
            outputs = non_max_suppression(outputs, conf_thres=conf_thres, iou_thres=nms_thres)

        sample_metrics += get_batch_statistics(outputs, targets, iou_threshold=iou_thres)

    if len(sample_metrics) == 0:  # No detections over whole validation set.
        print("---- No detections over whole validation set ----")
        return None

    # Concatenate sample statistics
    true_positives, pred_scores, pred_labels = [
        np.concatenate(x, 0) for x in list(zip(*sample_metrics))]
    metrics_output = ap_per_class(
        true_positives, pred_scores, pred_labels, labels)

    print_eval_stats(metrics_output, class_names, verbose)

    return metrics_output

In [21]:
def discriminator_step(
      discriminator,
      map_features, 
      labels
    ) -> None:

    """ 
    Discriminator step performed between the source and targer domain. 
    Input arguments:
      map_features: Tensor = feture map obtained from the feature extractor
      labels: Tensor = ground truth
    Return:
      Tensor = cross entropy loss between the prediction and the ground truth.
    """

    outputs = discriminator(map_features)
    outputs = outputs.view(mini_batch_size, -1)
    discriminator_loss = cross_entropy(outputs, labels)
    return discriminator_loss

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
# move models to GPU
model.to(device)
discriminator.to(device)

Discriminator(
  (net): Sequential(
    (0): GradientReversal(alpha=strtensor([1.], device='cuda:0'))
    (1): Linear(in_features=52, out_features=2048, bias=True)
    (2): ReLU()
    (3): Linear(in_features=2048, out_features=2048, bias=True)
    (4): ReLU()
    (5): Linear(in_features=2048, out_features=1, bias=True)
  )
)

In [24]:
# prepare target domain set
target_imgs_list = os.listdir(PATH_TO_TARGET)
t = transforms.Compose([
    transforms.Resize((416,416)),
    transforms.ToTensor()])
target_imgs = [t(Image.open(os.path.join(PATH_TO_TARGET, img))).float() for img in target_imgs_list]

In [25]:
# initialize wandb
run = wandb.init(project='yolo-uda', notes='initial config')
wandb.config = {
    "epochs": EPOCHS,
    "iou_thresh": IOU_THRESH,
    "conf_thresh": CONF_THRESH,
    "nms_thresh": NMS_THRESH,
    "alpha": alpha,
    "img_size": 416,
    "momentum": 0.9,
    "decay": 0.0005,
    "angle": 0,
    "saturation": 1.5,
    "exposure": 1.5,
    "hue": 0.1,
    "lr": 0.0001,
    "burn_in": 1000
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: earlranario. Use `wandb login --relogin` to force relogin


In [26]:
upsample_4 = Upsample(scale_factor=4, mode="nearest")
upsample_2 = Upsample(scale_factor=2, mode="nearest")
for epoch in range(1, EPOCHS+1):
    
    print("\n---- Training Model ----")

    # set to training mode
    model.train() # set yolo model to training mode
    discriminator.train() # set discriminator to training mode
    
    random.shuffle(target_imgs)
    
    for batch_i, (_, imgs, targets) in enumerate(tqdm.tqdm(dataloader, desc=f"Training Epoch {epoch}")):
        sample = random.sample(range(0, len(target_imgs)), mini_batch_size)
        batches_done = len(dataloader) * epoch + batch_i
        
        source_imgs = imgs.to(device)
        target_imgs = [target_imgs[i] for i in sample]
        target_imgs = torch.stack(target_imgs, dim=0).to(device)
        targets = targets.to(device)
        
        # run source pass, upsample features and calculate yolo loss
        source_outputs, source_features = model(source_imgs)
        source_features[0] = upsample_4(source_features[0])
        source_features[1] = upsample_2(source_features[1])
        yolo_loss, loss_components = compute_loss(source_outputs, targets, model)
        
        # run target pass upsample features
        zeros_label = torch.zeros(mini_batch_size, dtype=torch.long, device=device)
        ones_label = torch.ones(mini_batch_size, dtype=torch.long, device=device)
        target_outputs, target_features = model(target_imgs)
        target_features[0] = upsample_4(target_features[0])
        target_features[1] = upsample_2(target_features[1])
        
        # concatenate source and target features
        source_features = torch.cat(source_features, dim=1).to(device)
        target_features = torch.cat(target_features, dim=1).to(device)
        
        # discriminator step and calculate discriminator loss
        discriminator_source_loss = discriminator_step(discriminator, source_features, zeros_label)
        discriminator_target_loss = discriminator_step(discriminator, target_features, ones_label)
        discriminator_loss = discriminator_source_loss + discriminator_target_loss

        yolo_loss.backward(retain_graph=True) 
        discriminator_loss.backward()
        
        # run optimizer
        if batches_done % model.hyperparams['subdivisions'] == 0:
            # adapt learning rate
            lr = model.hyperparams['learning_rate']
            if batches_done < model.hyperparams['burn_in']:
                lr *= (batches_done / model.hyperparams['burn_in'])
            else:
                for threshold, value in model.hyperparams['lr_steps']:
                    if batches_done > threshold:
                        lr *= value
            ## log the learning rate here ##
            wandb.log({"lr": lr})
            # set leraning rate
            for g in optimizer.param_groups:
                g['lr'] = lr
                
            # Run optimizer
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()
    
        ## log progress here ##
        if VERBOSE:
            print(AsciiTable(
                    [
                        ["Type", "Value"],
                        ["IoU loss", float(loss_components[0])],
                        ["Object loss", float(loss_components[1])],
                        ["Class loss", float(loss_components[2])],
                        ["Loss", float(loss_components[3])],
                        ["Source loss", float(discriminator_source_loss)],
                        ["Target loss", float(discriminator_target_loss)],
                        ["Batch loss", to_cpu(loss).item()]
                    ]).table)
        wandb.log({
            "iou_loss": float(loss_components[0]),
            "obj_loss": float(loss_components[1]),
            "cls_loss": float(loss_components[2]),
            "loss": float(loss_components[3]),
            "src_loss": float(discriminator_source_loss),
            "trgt_loss": float(discriminator_target_loss)
            })
        model.seen += imgs.size(0)
        
    # save model to checkpoint file
    # if epoch % args.checkpoint_interval == 0:
    #     checkpoint_path = f"checkpoints/yolov3_ckpt_{epoch}.pth"
    #     print(f"---- Saving checkpoint to: '{checkpoint_path}' ----")
    #     torch.save(model.state_dict(), checkpoint_path)
    
    # evaluate
    if epoch % EVALUATE_INTERVAL == 0:
        print("\n---- Evaluating Model ----")
        # Evaluate the model on the validation set
        metrics_output = _evaluate(
            model,
            validation_dataloader,
            CLASS_NAMES,
            img_size=model.hyperparams['height'],
            iou_thres=IOU_THRESH,
            conf_thres=CONF_THRESH,
            nms_thres=NMS_THRESH,
            verbose=VERBOSE
        )

        if metrics_output is not None:
            precision, recall, AP, f1, ap_class = metrics_output
            wandb.log({
                "precision": precision.mean(),
                "recall": recall.mean(),
                "f1": f1.mean(),
                "mAP": AP.mean()
            })


---- Training Model ----


Training Epoch 1: 100%|██████████| 87/87 [01:25<00:00,  1.02it/s]



---- Evaluating Model ----
model training: False


Computing AP: 100%|██████████| 1/1 [00:00<00:00, 1854.25it/s]


---- mAP 0.00000 ----

---- Training Model ----


Training Epoch 2:  10%|█         | 9/87 [00:11<01:36,  1.24s/it]


KeyboardInterrupt: 

In [ ]:
# save model locally
SAVE = '/group/jmearlesgrp/data/yolo_grl_data/weights/baseline/20230914_yolov3_uda.pth'
torch.save(model.state_dict(), SAVE)

# save model to wandb
best_model = wandb.Artifact(f"20230914_yolov3_uda_{run.id}", type="model")
best_model.add_file("/group/jmearlesgrp/data/yolo_grl_data/weights/baseline/20230914_yolov3_uda.pth")
run.log_artifact(best_model)
run.link_artifact(best_model, "model-registry/yolo-uda")